# Audio Feature Analyses SIM 1

In [1]:
from scripts.load_data import check_and_load

## Data Loading

In [2]:
# Define paths
data_path = "../ground_truth_data/trimmed_videos"
#data_path = "../ground_truth_data"

frames_output_dir = "../ground_truth_data/frames"
audio_output_dir = "../ground_truth_data/audio"

annotations_path = "../ground_truth_data/trimmed_videos"
#annotations_path = "../ground_truth_data"

muppet_files = {
    "Muppets-02-01-01.avi": "GroundTruth_Muppets-02-01-01.csv",
    "Muppets-02-04-04.avi": "GroundTruth_Muppets-02-04-04.csv",
    "Muppets-03-04-03.avi": "GroundTruth_Muppets-03-04-03.csv",
}

In [4]:
# TODO: add code to overwrite frames or audio in case only one exists
annotations, audio_data, frames = check_and_load(data_path, frames_output_dir, audio_output_dir, annotations_path, muppet_files)

Frames and/or audio not extracted. Running setup script...
Loaded annotations for 'Muppets-02-01-01.avi'.
Loaded annotations for 'Muppets-02-04-04.avi'.
Loaded annotations for 'Muppets-03-04-03.avi'.


Extracting frames from Muppets-02-01-01.avi:   0%|          | 0/1500 [00:00<?, ?it/s]

Extracting frames from Muppets-02-01-01.avi: 100%|██████████| 1500/1500 [00:34<00:00, 43.38it/s]


Frames extracted from 'Muppets-02-01-01.avi' and saved to '../ground_truth_data/frames'.


Extracting frames from Muppets-02-04-04.avi: 100%|██████████| 1500/1500 [00:38<00:00, 39.13it/s]


Frames extracted from 'Muppets-02-04-04.avi' and saved to '../ground_truth_data/frames'.


Extracting frames from Muppets-03-04-03.avi: 100%|██████████| 1500/1500 [00:38<00:00, 39.09it/s]


Frames extracted from 'Muppets-03-04-03.avi' and saved to '../ground_truth_data/frames'.
Extracted and aligned audio segments for 'Muppets-02-01-01.avi'. Full audio saved to '../ground_truth_data/audio\full_audio'.
Extracted and aligned audio segments for 'Muppets-02-04-04.avi'. Full audio saved to '../ground_truth_data/audio\full_audio'.
Extracted and aligned audio segments for 'Muppets-03-04-03.avi'. Full audio saved to '../ground_truth_data/audio\full_audio'.
Loading audio segments...
Loaded audio segments for 3 videos.
Loaded audio segments for 3 videos.
Video 0 has 1500 audio segments.
Video 1 has 1500 audio segments.
Video 2 has 1500 audio segments.
Loaded frames for 3 videos.
Number of videos with frames: 3
Video 0 has 1500 frames.
Video 1 has 1500 frames.
Video 2 has 1500 frames.
